In [ ]:
!pip install -q transformers evaluate torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.0 MB/s eta 0:00:00


In [ ]:
# import libraries
import torch
import nltk
import evaluate
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from torchmetrics.text import BLEUScore
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text import SacreBLEUScore
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from nltk.translate.gleu_score import sentence_gleu



import json
from datasets import load_dataset

In [ ]:
# device config
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

# Evaluation metrics

In [ ]:
bleu = BLEUScore()
rouge = ROUGEScore()
sacre_bleu = SacreBLEUScore()

# Model config

In [ ]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Falconsai/question_answering_v2")
# inputs = tokenizer(question, context, return_tensors="pt")

model = AutoModelForQuestionAnswering.from_pretrained("Falconsai/question_answering_v2")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

# PrivacyQA Dataset

In [ ]:
# read dataset
test_privacy_df = pd.read_csv("/content/policy_test_data.csv", sep="\t")
test_privacy_df = test_privacy_df[['Query', 'Segment']]

test_privacy_df = test_privacy_df.drop_duplicates(keep=False)

In [ ]:
# mkae datasetclass
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data.iloc[idx]['Query']
        context = self.data.iloc[idx]['Segment']

        # Tokenize question and context with padding and truncation
        inputs = self.tokenizer(question, context, return_tensors="pt", padding=True, truncation=True, max_length=self.max_length)

        return inputs, context

# Create an instance of your custom dataset
custom_dataset = CustomDataset(test_privacy_df, tokenizer)

In [ ]:
# make dataloader
BATCH_SIZE = 1
dataloader = DataLoader(custom_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
bleu_score = 0
sacre_bleu_score = 0
gleu_score = 0

rouge1_fmeasure = 0
rouge1_precision = 0
rouge1_recall = 0

rouge2_fmeasure = 0
rouge2_precision = 0
rouge2_recall = 0

rougeL_fmeasure = 0
rougeL_precision = 0
rougeL_recall = 0



model = model.to(device)
j = 0
for batch, context in dataloader:
    with torch.no_grad():
        batch['input_ids'] = batch['input_ids'].squeeze(dim=0)
        batch['attention_mask'] = batch['attention_mask'].squeeze(dim=0)

        batch = batch.to(device)
        outputs = model(**batch)
        answer_start_indices = outputs.start_logits.argmax(dim=1)
        answer_end_indices = outputs.end_logits.argmax(dim=1)
        for i in range(BATCH_SIZE):

            start_idx = answer_start_indices[i].item()
            end_idx = answer_end_indices[i].item()
            predict_answer_tokens = batch.input_ids[i, start_idx : end_idx + 1]
            predicted_answer = tokenizer.decode(predict_answer_tokens)


            bleu_results = bleu([predicted_answer], [[context[0]]])
            rouge_result = rouge(predicted_answer, context[0])
            sacre_bleu_result = sacre_bleu([predicted_answer], [[context[0]]])
            gleu_score_result = sentence_gleu(predicted_answer.split(), context[0].split())


            bleu_score += bleu_results
            sacre_bleu_score += sacre_bleu_result
            gleu_score += gleu_score_result

            rouge1_fmeasure += rouge_result['rouge1_fmeasure']
            rouge1_precision += rouge_result['rouge1_precision']
            rouge1_recall += rouge_result['rouge1_recall']

            rouge2_fmeasure += rouge_result['rouge2_fmeasure']
            rouge2_precision += rouge_result['rouge2_precision']
            rouge2_recall += rouge_result['rouge2_recall']

            rougeL_fmeasure += rouge_result['rougeL_fmeasure']
            rougeL_precision += rouge_result['rougeL_precision']
            rougeL_recall += rouge_result['rougeL_recall']


            # print("Predicted Answer:", predicted_answer)
        if j == 7000:
          break
        j += 1

# bleu_score /= len(dataloader)
# rouge_score /= len(dataloader)
# sacre_bleu_score /= len(dataloader)
# gleu_score /= len(dataloader)

bleu_score /= 7000
sacre_bleu_score /= 7000
gleu_score /= 7000

rouge1_fmeasure /= 7000
rouge1_precision /= 7000
rouge1_recall /= 7000

rouge2_fmeasure /= 7000
rouge2_precision /= 7000
rouge2_recall /= 7000

rougeL_fmeasure /= 7000
rougeL_precision /= 7000
rougeL_recall /= 7000

In [ ]:
print(f"BLEU: {bleu_score:.4f}")
print(f"SACRE_BLEU: {sacre_bleu_score:.4f}")
print(f"GLEU: {gleu_score:.4f}")
print()

print(f"ROUGH1 Fmeasure: {rouge1_fmeasure:.4f}")
print(f"ROUGH1 Precision: {rouge1_precision:.4f}")
print(f"ROUGH1 Recall: {rouge1_recall:.4f}")
print()

print(f"ROUGH2 Fmeasure: {rouge2_fmeasure:.4f}")
print(f"ROUGH2 Precision: {rouge2_precision:.4f}")
print(f"ROUGH2 Recall: {rouge2_recall:.4f}")
print()

print(f"ROUGHL Fmeasure: {rougeL_fmeasure:.4f}")
print(f"ROUGHL Precision: {rougeL_precision:.4f}")
print(f"ROUGHL Recall: {rougeL_recall:.4f}")

BLEU: 0.1727
SACRE_BLEU: 0.1733
GLEU: 0.0025

ROUGH1 Fmeasure: 0.4127
ROUGH1 Precision: 0.7135
ROUGH1 Recall: 0.3569

ROUGH2 Fmeasure: 0.3783
ROUGH2 Precision: 0.6500
ROUGH2 Recall: 0.3318

ROUGHL Fmeasure: 0.4116
ROUGHL Precision: 0.7120
ROUGHL Recall: 0.3559


## Text Genration Example

In [ ]:
question = test_privacy_df.iloc[0]['Query']
context = test_privacy_df.iloc[0]['Segment']

inputs = tokenizer(question, context, return_tensors="pt")
inputs = inputs.to(device)
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
pred_text = tokenizer.decode(predict_answer_tokens)

print(f"Question: {question}\n")
print("Answer")
print(f"Predition Text: {pred_text}")
print(f"Reference Text: {context}")

Question: is my chat here with the platform confidential?

Answer
Predition Text: at fiverr we care about your privacy
Reference Text:   At Fiverr we care about your privacy.


# PolicyQA Dataset

In [ ]:
def preprocess_to_csv(file_path, split = 'train'):

  with open(file_path, "r") as f:
    data = json.load(f)

  new_data = []

  for item in data['data']:
      for paragraph in item['paragraphs']:

          context = paragraph['context']

          for qa in paragraph['qas']:
              entry = {
                  'question': qa['question'],
                  'answer': qa['answers'][0]['text'] if qa['answers'] else "NA",
                  'answer_start' : qa['answers'][0]['answer_start'],
                  'answer_end' : qa['answers'][0]['answer_start'] + len(qa['answers'][0]['text']),
                  'context': context,
                  'id': qa['id'],
              }
              new_data.append(entry)

  df = pd.DataFrame(new_data)
  df.to_csv(f"{split}.csv", index=False)
  data = pd.read_csv(f"{split}.csv")

  # dataset = load_dataset("csv", data_files = {"data": f"{split}.csv"})

  return data

In [ ]:
test_data = preprocess_to_csv('/content/test.json', split = 'test')

In [ ]:
test_data_df = test_data[['question', 'answer']]

In [ ]:
test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4152 entries, 0 to 4151
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  4152 non-null   object
 1   answer    4151 non-null   object
dtypes: object(2)
memory usage: 65.0+ KB


In [ ]:
test_data_df = test_data_df.drop_duplicates(keep=False)
test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3776 entries, 0 to 4151
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  3776 non-null   object
 1   answer    3775 non-null   object
dtypes: object(2)
memory usage: 88.5+ KB


In [ ]:
test_data_df.isnull().sum()

question    0
answer      1
dtype: int64

In [ ]:
test_data_df.dropna(inplace=True)
test_data_df.isnull().sum()

question    0
answer      0
dtype: int64

In [ ]:
test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3775 entries, 0 to 4151
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  3775 non-null   object
 1   answer    3775 non-null   object
dtypes: object(2)
memory usage: 88.5+ KB


In [ ]:
test_data_df.head()

,question,answer
0,Do you take the user's opinion before or after...,"Last Updated on May 22, 2015"
1,How do you inform all the users upon policy ch...,"Last Updated on May 22, 2015"
2,What types of choice options available to the ...,If you disagree with any part of this Privacy ...
3,What are the users' choices for the data used ...,The Privacy Policy applies to your access and ...
4,Does the company collect user's information di...,access and use of the NOTMC Websites (as defin...


In [ ]:
# mkae datasetclass
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data.iloc[idx]['question']
        context = self.data.iloc[idx]['answer']

        # Tokenize question and context with padding and truncation
        inputs = self.tokenizer(question, context, return_tensors="pt", padding=True, truncation=True, max_length=self.max_length)

        return inputs, context

# Create an instance of your custom dataset
custom_dataset = CustomDataset(test_data_df, tokenizer)

In [ ]:
# make dataloader
BATCH_SIZE = 1
dataloader = DataLoader(custom_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
bleu_score = 0
sacre_bleu_score = 0
gleu_score = 0

rouge1_fmeasure = 0
rouge1_precision = 0
rouge1_recall = 0

rouge2_fmeasure = 0
rouge2_precision = 0
rouge2_recall = 0

rougeL_fmeasure = 0
rougeL_precision = 0
rougeL_recall = 0



model = model.to(device)
j = 0
for batch, context in dataloader:
    with torch.no_grad():
        batch['input_ids'] = batch['input_ids'].squeeze(dim=0)
        batch['attention_mask'] = batch['attention_mask'].squeeze(dim=0)

        batch = batch.to(device)
        outputs = model(**batch)
        answer_start_indices = outputs.start_logits.argmax(dim=1)
        answer_end_indices = outputs.end_logits.argmax(dim=1)
        for i in range(BATCH_SIZE):

            start_idx = answer_start_indices[i].item()
            end_idx = answer_end_indices[i].item()
            predict_answer_tokens = batch.input_ids[i, start_idx : end_idx + 1]
            predicted_answer = tokenizer.decode(predict_answer_tokens)


            bleu_results = bleu([predicted_answer], [[context[0]]])
            rouge_result = rouge(predicted_answer, context[0])
            sacre_bleu_result = sacre_bleu([predicted_answer], [[context[0]]])
            gleu_score_result = sentence_gleu(predicted_answer.split(), context[0].split())


            bleu_score += bleu_results
            sacre_bleu_score += sacre_bleu_result
            gleu_score += gleu_score_result

            rouge1_fmeasure += rouge_result['rouge1_fmeasure']
            rouge1_precision += rouge_result['rouge1_precision']
            rouge1_recall += rouge_result['rouge1_recall']

            rouge2_fmeasure += rouge_result['rouge2_fmeasure']
            rouge2_precision += rouge_result['rouge2_precision']
            rouge2_recall += rouge_result['rouge2_recall']

            rougeL_fmeasure += rouge_result['rougeL_fmeasure']
            rougeL_precision += rouge_result['rougeL_precision']
            rougeL_recall += rouge_result['rougeL_recall']


            # print("Predicted Answer:", predicted_answer)
        # if j == 7000:
        #   break
        # j += 1

# bleu_score /= len(dataloader)
# rouge_score /= len(dataloader)
# sacre_bleu_score /= len(dataloader)
# gleu_score /= len(dataloader)

bleu_score /= len(dataloader)
sacre_bleu_score /= len(dataloader)
gleu_score /= len(dataloader)

rouge1_fmeasure /= len(dataloader)
rouge1_precision /= len(dataloader)
rouge1_recall /= len(dataloader)

rouge2_fmeasure /= len(dataloader)
rouge2_precision /= len(dataloader)
rouge2_recall /= len(dataloader)

rougeL_fmeasure /= len(dataloader)
rougeL_precision /= len(dataloader)
rougeL_recall /= len(dataloader)

In [ ]:
32print(f"BLEU: {bleu_score:.4f}")
print(f"SACRE_BLEU: {sacre_bleu_score:.4f}")
print(f"GLEU: {gleu_score:.4f}")
print()

print(f"ROUGH1 Fmeasure: {rouge1_fmeasure:.4f}")
print(f"ROUGH1 Precision: {rouge1_precision:.4f}")
print(f"ROUGH1 Recall: {rouge1_recall:.4f}")
print()

print(f"ROUGH2 Fmeasure: {rouge2_fmeasure:.4f}")
print(f"ROUGH2 Precision: {rouge2_precision:.4f}")
print(f"ROUGH2 Recall: {rouge2_recall:.4f}")
print()

print(f"ROUGHL Fmeasure: {rougeL_fmeasure:.4f}")
print(f"ROUGHL Precision: {rougeL_precision:.4f}")
print(f"ROUGHL Recall: {rougeL_recall:.4f}")

BLEU: 0.1553
SACRE_BLEU: 0.1631
GLEU: 0.0020

ROUGH1 Fmeasure: 0.4705
ROUGH1 Precision: 0.6712
ROUGH1 Recall: 0.4498

ROUGH2 Fmeasure: 0.4049
ROUGH2 Precision: 0.5854
ROUGH2 Recall: 0.3898

ROUGHL Fmeasure: 0.4701
ROUGHL Precision: 0.6707
ROUGHL Recall: 0.4492


## Text Genration Example

In [ ]:
question = test_data_df.iloc[1]['question']
context = test_data_df.iloc[1]['answer']

inputs = tokenizer(question, context, return_tensors="pt")
inputs = inputs.to(device)
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
pred_text = tokenizer.decode(predict_answer_tokens)

print(f"Question: {question}\n")
print("Answer")
print(f"Predition Text: {pred_text}")
print(f"Reference Text: {context}")

Question: How do you inform all the users upon policy changes?

Answer
Predition Text: may 22, 2015
Reference Text: Last Updated on May 22, 2015
